In [63]:
!pip3 install pycaret
!pip3 install yahoofinancials
!pip3 install numpy
!pip3 install pandas
!pip3 install datetime
!pip3 install matplotlib
!pip3 install yahoofinancials
!pip3 install openpyxl

In [64]:
#Importing Libraries
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from yahoofinancials import YahooFinancials

In [65]:
ticker_details = pd.read_excel("Ticker List.xlsx")
ticker = ticker_details['Ticker'].to_list()
names = ticker_details['Description'].to_list()

df_ticker = pd.DataFrame(ticker)
df_ticker

,0
0,GC=F
1,SI=F
2,CL=F
3,^GSPC
4,^RUT
5,ZN=F
6,ZT=F
7,PL=F
8,HG=F
9,DX=F


In [66]:
print(names)
df_names = pd.DataFrame(names)

df_names

['Gold', 'Silver', 'Crude Oil', 'S&P500', 'Russel 2000 Index', '10 Yr US T-Note futures', '2 Yr US T-Note Futures', 'Platinum', 'Copper', 'Dollar Index', 'Volatility Index', 'Soybean', 'MSCI EM ETF', 'Euro USD', 'Euronext100', 'Nasdaq']


,0
0,Gold
1,Silver
2,Crude Oil
3,S&P500
4,Russel 2000 Index
5,10 Yr US T-Note futures
6,2 Yr US T-Note Futures
7,Platinum
8,Copper
9,Dollar Index


In [67]:
#Preparing Date Range
end_date = datetime.strftime(datetime.today(), '%Y-%m-%d')
start_date = "2019-01-01"
date_range = pd.bdate_range(start=start_date, end=end_date)
values = pd.DataFrame({'Date': date_range})
values['Date'] = pd.to_datetime(values['Date'])

print(values)

           Date
0    2019-01-01
1    2019-01-02
2    2019-01-03
3    2019-01-04
4    2019-01-07
...         ...
1340 2024-02-20
1341 2024-02-21
1342 2024-02-22
1343 2024-02-23
1344 2024-02-26

[1345 rows x 1 columns]


In [68]:
YahooFinancials(i)
YahooFinancials

yahoofinancials.yf.YahooFinancials

In [76]:
#Extracting Data from Yahoo Finance and Adding them to Values table using date as key
for i in ticker:
	raw_data = YahooFinancials(i)
	raw_data = raw_data.get_historical_price_data(start_date, end_date, "daily")

In [77]:
# print(raw_data)
df_raw_data = pd.DataFrame(raw_data)

df_raw_data

,^IXIC
currency,USD
eventsData,{}
firstTradeDate,"{'formatted_date': '1971-02-05', 'date': 34612..."
instrumentType,INDEX
prices,"[{'date': 1546439400, 'high': 6693.7099609375,..."
timeZone,{'gmtOffset': -18000}


In [74]:
#Extracting Data from Yahoo Finance and Adding them to Values table using date as key
for i in ticker:
	raw_data = YahooFinancials(i)
	raw_data = raw_data.get_historical_price_data(start_date, end_date, "daily")
	df_none = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=None)
	df_raw_data_adjusted = pd.DataFrame(raw_data[i]['prices'])[['formatted_date', 'adjclose']]
	df_raw_data_adjusted.columns = ['Date1', i]
	df_raw_data_adjusted['Date1'] = pd.to_datetime(df_raw_data_adjusted['Date1'])
	values = values.merge(df_raw_data_adjusted, how='left', left_on='Date', right_on='Date1')
	values = values.drop(labels='Date1', axis=1)
	print(values)

df_raw_data = pd.DataFrame(raw_data)

df_raw_data
#print(values)
# df

           Date         GC=F
0    2019-01-01          NaN
1    2019-01-02  1281.000000
2    2019-01-03  1291.800049
3    2019-01-04  1282.699951
4    2019-01-07  1286.800049
...         ...          ...
1340 2024-02-20  2027.500000
1341 2024-02-21  2022.300049
1342 2024-02-22  2019.699951
1343 2024-02-23  2038.599976
1344 2024-02-26          NaN

[1345 rows x 2 columns]
           Date         GC=F       SI=F
0    2019-01-01          NaN        NaN
1    2019-01-02  1281.000000  15.542000
2    2019-01-03  1291.800049  15.706000
3    2019-01-04  1282.699951  15.695000
4    2019-01-07  1286.800049  15.669000
...         ...          ...        ...
1340 2024-02-20  2027.500000  23.106001
1341 2024-02-21  2022.300049  22.851000
1342 2024-02-22  2019.699951  22.764999
1343 2024-02-23  2038.599976  22.969000
1344 2024-02-26          NaN        NaN

[1345 rows x 3 columns]
           Date         GC=F       SI=F       CL=F
0    2019-01-01          NaN        NaN        NaN
1    2019-01-02  128

TypeError: 'NoneType' object is not subscriptable

In [ ]:
#Renaming columns to represent instrument names rather than their ticker codes for ease of readability
# names.insert(0, 'Date')
values.columns = names
names

In [ ]:
#Front filling the NaN values in the data set
values = values.fillna(method="ffill", axis=0)
values = values.fillna(method="bfill", axis=0)

In [ ]:
# Co-ercing numeric type to all columns except Date
cols = values.columns.drop('Date')
values[cols] = values[cols].apply(pd.to_numeric, errors='coerce').round(decimals=1)
imp = ['Gold', 'Silver', 'Crude Oil', 'S&P500', 'MSCI EM ETF']

In [ ]:
# Calculating Short term -Historical Returns
change_days = [1, 3, 5, 14, 21]

data = pd.DataFrame(data=values['Date'])
for i in change_days:
	x = values[cols].pct_change(periods=i).add_suffix("-T-" + str(i))
	data = pd.concat(objs=(data, x), axis=1)
	x = []

# Calculating Long term Historical Returns
change_days = [60, 90, 180, 250]

for i in change_days:
	x = values[imp].pct_change(periods=i).add_suffix("-T-" + str(i))
	data = pd.concat(objs=(data, x), axis=1)
	x = []

#Calculating Moving averages for Gold
moving_avg = pd.DataFrame(values['Date'], columns=['Date'])
moving_avg['Date'] = pd.to_datetime(moving_avg['Date'], format='%Y-%b-%d')
moving_avg['Gold/15SMA'] = (values['Gold'] / (values['Gold'].rolling(window=15).mean())) - 1
moving_avg['Gold/30SMA'] = (values['Gold'] / (values['Gold'].rolling(window=30).mean())) - 1
moving_avg['Gold/60SMA'] = (values['Gold'] / (values['Gold'].rolling(window=60).mean())) - 1
moving_avg['Gold/90SMA'] = (values['Gold'] / (values['Gold'].rolling(window=90).mean())) - 1
moving_avg['Gold/180SMA'] = (values['Gold'] / (values['Gold'].rolling(window=180).mean())) - 1
moving_avg['Gold/90EMA'] = (values['Gold'] / (values['Gold'].ewm(span=90, adjust=True, ignore_na=True).mean())) - 1
moving_avg['Gold/180EMA'] = (values['Gold'] / (values['Gold'].ewm(span=180, adjust=True, ignore_na=True).mean())) - 1
moving_avg = moving_avg.dropna(axis=0)

#Merging Moving Average values to the feature space

data['Date'] = pd.to_datetime(data['Date'], format='%Y-%b-%d')
data = pd.merge(left=data, right=moving_avg, how='left', on='Date')
data = data[data['Gold-T-250'].notna()]
prediction_data = data.copy()

In [ ]:
from pycaret.classification import *

#Loading the stored model
classifier_22 = load_model("22D Classifier");

#Making Predictions
prediction = predict_model(classifier_22, data=prediction_data)


In [ ]:
prediction[['Date', 'Label', 'Score']].tail(10)